In [1]:
import numpy as np
import pandas as pd
import os

from bmtool.util.util import load_nodes_from_paths

network_dir = 'network'

# Must set to the same as in build_network.py
prob_ratio = 2  # ratio of connection probability within vs. between assemblies

wsyn_ratio = 2  # ratio of mean synaptic weigth within vs. between assemblies

### Get number of population nodes

In [2]:
network = 'cortex'
node_files = [{"nodes_file": "cortex_nodes.h5", "node_types_file": "cortex_node_types.csv"}]
for d in node_files:
    for k, f in d.items():
        d[k] = os.path.join(network_dir, f)

node_df = load_nodes_from_paths(node_files)[network]

PN_pop_names = ['CP', 'CS']
num_CP, num_CS = [sum(node_df['pop_name'] == p) for p in PN_pop_names]
num_PN = num_CP + num_CS

### Get edges info
#### Apply scaling to both core-to-core and shell-to-core

In [3]:
# edges_file = 'cortex_cortex_edge_types.csv'
edges_file = 'shell_cortex_edge_types.csv'

edges_df = pd.read_csv(os.path.join(network_dir, edges_file), delimiter=' ', index_col='edge_type_id')

In [4]:
def get_query(s):
    condstr = s.split('&')
    cond = {}
    for c in condstr:
        key, val = c.split('==')
        cond[key] = eval(val[1:-1])
    return cond

PN_conn = {}
src_trg_query = ['source_query', 'target_query']
for i, row in edges_df.iterrows():
    conn = {}
    for query in src_trg_query:
        s = row[query]
        if 'assembly_id' in s:
            for k, v in get_query(s).items():
                conn[(query, k)] = v
    if conn:
        PN_conn[i] = conn

PN_conn = pd.DataFrame.from_dict(PN_conn, orient='index')

### Estimate scaling factor of synaptic weights to maintain overall input to each cell

In [5]:
n_assemblies = PN_conn['target_query', 'assembly_id'].unique().size

# Scaling factor of connection probabilities for PN assemblies
# proportion of presynapstic cells in an assembly
assy_prop = (num_PN / n_assemblies - 1) / (num_PN - 1)
# Scale connection probability according to assembly
# If n_assemblies is 1, all PNs belong to the same assembly. Scaling factor is 1.
p_diff_scale = 1 / (assy_prop * (prob_ratio - 1) + 1)  # between assemblies
p_same_scale = p_diff_scale * prob_ratio  # within the same assembly

# Scaling factor of mean synaptic weigth for PN assemblies
g_diff_scale = 1 / (assy_prop * p_same_scale * wsyn_ratio + (1 - assy_prop) * p_diff_scale)  # between assemblies
g_same_scale = wsyn_ratio * g_diff_scale  # within the same assembly

In [6]:
same_assy = PN_conn['source_query', 'assembly_id'] == PN_conn['target_query', 'assembly_id']
diff_assy = edges_df.loc[same_assy.index].loc[~same_assy].index
same_assy = edges_df.loc[same_assy.index].loc[same_assy].index

scale_cols = ['syn_weight', 'weight_sigma']
edges_scaled_df = edges_df.copy()
edges_scaled_df.loc[diff_assy, scale_cols] *= g_diff_scale
edges_scaled_df.loc[same_assy, scale_cols] *= g_same_scale

display(edges_scaled_df)

,target_query,source_query,weight_sigma,model_template,afferent_section_pos,sigma_upper_bound,dynamics_params,afferent_section_id,syn_weight,weight_function
edge_type_id,,,,,,,,,,
154,pop_name=='['CP']'&assembly_id=='0',pop_name=='['CP']'&assembly_id=='0',1.066733,AMPA_NMDA_STP,0.4,3.0,CP2CP.json,1,1.333417,lognormal_weight
155,pop_name=='['CP']'&assembly_id=='1',pop_name=='['CP']'&assembly_id=='0',0.533367,AMPA_NMDA_STP,0.4,3.0,CP2CP.json,1,0.666708,lognormal_weight
156,pop_name=='['CP']'&assembly_id=='2',pop_name=='['CP']'&assembly_id=='0',0.533367,AMPA_NMDA_STP,0.4,3.0,CP2CP.json,1,0.666708,lognormal_weight
157,pop_name=='['CP']'&assembly_id=='0',pop_name=='['CP']'&assembly_id=='1',0.533367,AMPA_NMDA_STP,0.4,3.0,CP2CP.json,1,0.666708,lognormal_weight
158,pop_name=='['CP']'&assembly_id=='1',pop_name=='['CP']'&assembly_id=='1',1.066733,AMPA_NMDA_STP,0.4,3.0,CP2CP.json,1,1.333417,lognormal_weight
159,pop_name=='['CP']'&assembly_id=='2',pop_name=='['CP']'&assembly_id=='1',0.533367,AMPA_NMDA_STP,0.4,3.0,CP2CP.json,1,0.666708,lognormal_weight
160,pop_name=='['CP']'&assembly_id=='0',pop_name=='['CP']'&assembly_id=='2',0.533367,AMPA_NMDA_STP,0.4,3.0,CP2CP.json,1,0.666708,lognormal_weight
161,pop_name=='['CP']'&assembly_id=='1',pop_name=='['CP']'&assembly_id=='2',0.533367,AMPA_NMDA_STP,0.4,3.0,CP2CP.json,1,0.666708,lognormal_weight
162,pop_name=='['CP']'&assembly_id=='2',pop_name=='['CP']'&assembly_id=='2',1.066733,AMPA_NMDA_STP,0.4,3.0,CP2CP.json,1,1.333417,lognormal_weight


### Save scaled edge file

In [7]:
edges_scaled_file = edges_file.replace('.csv', '_scaled.csv')
edges_scaled_df.to_csv(os.path.join(network_dir, edges_scaled_file), sep=' ', na_rep='NULL')